In [2]:
## imports dependencies
import pandas as pd
import os

In [3]:
## creates file path for raw data
raw_data_file_path = os.path.join('raw_data', 'ncaa_mens_march_madness_historical_results.csv')

In [24]:
## create a team-game level dataframe
## includes original statistics, a game id, a  flag for win-loss, and a seed matchup id
march_madness_df = pd.read_csv(raw_data_file_path)
march_madness_df.reset_index(inplace=True)
final_columns = ['game_id', 'date', 'round', 'region', 'seed', 'team',
                 'score', 'opponent_seed', 'opponent', 'opponent_score', 'overtime']
winner_df = march_madness_df.copy()
winner_df.columns = final_columns
loser_df = march_madness_df.copy()
loser_df.columns = ['game_id', 'date', 'round', 'region', 'opponent_seed', 'opponent',
                    'opponent_score', 'seed', 'team', 'score', 'overtime']
loser_df = loser_df[final_columns]
team_game_df = winner_df.append(loser_df, ignore_index=True)
team_game_df = team_game_df.sort_values('game_id')
team_game_df.reset_index(drop=True, inplace=True)
team_game_df['score_diff'] = team_game_df['score'] - team_game_df['opponent_score']
team_game_df["win"] = [1 if score_diff > 0 else 0 for score_diff in team_game_df['score_diff']]
team_game_df['seed_id'] = ''
for  index, row in team_game_df.iterrows():
    seed = team_game_df.loc[index]['seed']
    opponent_seed = team_game_df.loc[index]['opponent_seed']
    if seed < opponent_seed:
        seed_id = f'{seed}_{opponent_seed}_fav'
    elif seed > opponent_seed:
        seed_id = f'{opponent_seed}_{seed}_dog'
    else:
        seed_id = f'{seed}_{seed}'
    team_game_df.loc[index, 'seed_id'] = seed_id
team_game_df['year'] = [int(date.split("-")[0]) for date in team_game_df['date']]
team_game_df['overtime'].fillna(0, inplace=True)
team_game_df['overtime'] = [0 if ot_count == 0 else int(ot_count[:-3]) for ot_count in team_game_df['overtime']]
team_game_df

game_id        date                  round     region  seed  \
0           0  1985-03-14            Round of 64       East     1   
1           0  1985-03-14            Round of 64       East    16   
2           1  1985-03-14            Round of 64       East     4   
3           1  1985-03-14            Round of 64       East    13   
4           2  1985-03-14            Round of 64       East     5   
5           2  1985-03-14            Round of 64       East    12   
6           3  1985-03-14            Round of 64       East     9   
7           3  1985-03-14            Round of 64       East     8   
8           4  1985-03-14            Round of 64    Midwest    16   
9           4  1985-03-14            Round of 64    Midwest     1   
10          5  1985-03-14            Round of 64    Midwest    13   
11          5  1985-03-14            Round of 64    Midwest     4   
12          6  1985-03-14            Round of 64    Midwest     5   
13          6  1985-03-14            Round of 64    Midwest    12   
14          7  1985-03-14            Round of 64    Midwest     8   
15          7  1985-03-14            Round of 64    Midwest     9   
16          8  1985-03-14            Round of 64  Southeast    15   
17          8  1985-03-14            Round of 64  Southeast     2   
18          9  1985-03-14            Round of 64  Southeast    14   
19          9  1985-03-14            Round of 64  Southeast     3   
20         10  1985-03-14            Round of 64  Southeast     7   
21         10  1985-03-14            Round of 64  Southeast    10   
22         11  1985-03-14            Round of 64  Southeast     6   
23         11  1985-03-14            Round of 64  Southeast    11   
24         12  1985-03-14            Round of 64       West    16   
25         12  1985-03-14            Round of 64       West     1   
26         13  1985-03-14            Round of 64       West    13   
27         13  1985-03-14            Round of 64       West     4   
28         14  1985-03-14            Round of 64       West     9   
29         14  1985-03-14            Round of 64       West     8   
...       ...         ...                    ...        ...   ...   
4070     2035  2016-03-24          Sweet Sixteen      South     5   
4071     2035  2016-03-24          Sweet Sixteen      South     1   
4072     2036  2016-03-24          Sweet Sixteen      South     2   
4073     2036  2016-03-24          Sweet Sixteen      South     3   
4074     2037  2016-03-24          Sweet Sixteen       West     4   
4075     2037  2016-03-24          Sweet Sixteen       West     1   
4076     2038  2016-03-24          Sweet Sixteen       West     3   
4077     2038  2016-03-24          Sweet Sixteen       West     2   
4078     2039  2016-03-25          Sweet Sixteen       East     5   
4079     2039  2016-03-25          Sweet Sixteen       East     1   
4080     2040  2016-03-25          Sweet Sixteen       East     7   
4081     2040  2016-03-25          Sweet Sixteen       East     6   
4082     2041  2016-03-25          Sweet Sixteen    Midwest     4   
4083     2041  2016-03-25          Sweet Sixteen    Midwest     1   
4084     2042  2016-03-25          Sweet Sixteen    Midwest    11   
4085     2042  2016-03-25          Sweet Sixteen    Midwest    10   
4086     2043  2016-03-26            Elite Eight      South     1   
4087     2043  2016-03-26            Elite Eight      South     2   
4088     2044  2016-03-26            Elite Eight       West     2   
4089     2044  2016-03-26            Elite Eight       West     1   
4090     2045  2016-03-27            Elite Eight       East     6   
4091     2045  2016-03-27            Elite Eight       East     1   
4092     2046  2016-03-27            Elite Eight    Midwest     1   
4093     2046  2016-03-27            Elite Eight    Midwest    10   
4094     2047  2016-04-02    National Semifinals        NaN    10   
4095     2047  2016-04-02    National Semifinals        NaN     1   
4096

In [26]:
## saves new dataframe to csv
team_game_level_file_path = os.path.join('raw_data', 'team_game_level_data.csv')
team_game_df.to_csv(team_game_level_file_path, index=False)